In [1]:
using Pkg
Pkg.activate("D://ATIS3//Project.toml") # change path 
Pkg.instantiate()

 Activating environment at `D:\ATIS3\Project.toml`


In [2]:
using JuMP, LinearAlgebra, Plots, StatsPlots, CPLEX, Statistics, CSV, DataFrames

In [11]:
#read scenario data
scenarios_hexagons = CSV.read("scenarios_multinomial.csv", DataFrame)
scenarios_prices = CSV.read("priceScenarions3.csv", DataFrame)

SOC = 0.33
hexagons = names(scenarios_hexagons)[1:length(names(scenarios_hexagons))-1] # vector of hexagons (strings)
hours = names(scenarios_prices)[3:length(names(scenarios_prices))] #vector of hours for charging (strings)
car_kWh = 30 # capacity of each car

#demand mxn (m=hexagon, n=scenario) in kWh summe über car*max_kWh*(1-SOC)
demand_cars = transpose(convert(Matrix,scenarios_hexagons[:,hexagons]))
demand_kWh = demand_cars.*car_kWh.*(1-SOC)

#prices mxn (m=hours, n=scenario)
prices = transpose(convert(Matrix,scenarios_prices[:,hours]))

probability_hexagons = transpose(scenarios_hexagons.probs) # probabilities for each scenario (#cars per hexagon)
probability_prices = transpose(scenarios_prices.Probability) # probabilities for each scenario (timeseries prices)

P = [3.7, 11, 22] # different charger types in kW
charger_cost_fix = [1500, 1700, 1900] # investment cost for charger 3 kW, 7 kW und 11 kW in €
charger_cost_annuity = charger_cost_fix./(5*365)
charger_cost = charger_cost_annuity./P # investment cost for charger 3 kW, 7 kW und 11 kW in €/kW
opportunity_cost = 0 # cost of demand not served

alpha = 0.05
beta = 0

0

## Build the scenario tree

In [16]:
## Build Probabilities
comb_probability = []
for prob_d in probability_hexagons
    for prob_p in probability_prices
        push!(comb_probability,prob_d*prob_p)
    end
end

## Build Demand kWh and Demand Cars
num_hex = size(demand_kWh,1)
num_Scen_Dem = size(demand_kWh,2)
num_Scen_Pri = size(prices,2)

comb_demand_kWh = zeros((num_hex, num_Scen_Dem*num_Scen_Pri))
comb_demand_cars = zeros((num_hex, num_Scen_Dem*num_Scen_Pri))
counter = 0

for num_d in 1:num_Scen_Dem
    for num_p in 1:num_Scen_Pri
        counter += 1
        comb_demand_kWh[1:num_hex,counter] = demand_kWh[1:num_hex,num_d]
        comb_demand_cars[1:num_hex,counter] = demand_cars[1:num_hex,num_d]
    end
end

## Build Prices
num_Scen_Dem = size(demand_kWh,2)
num_Scen_Pri = size(prices,2)

comb_prices = zeros((length(hours), num_Scen_Dem*num_Scen_Pri))
counter = 0

for num_d in 1:num_Scen_Dem
    for num_p in 1:num_Scen_Pri
        counter += 1
        comb_prices[:,counter] = prices[1:length(hours),num_p]
    end
end

probability = convert(Array{Float64,1}, comb_probability)
demand_kWh = comb_demand_kWh
demand_cars = comb_demand_cars
prices = comb_prices


7×1764 Array{Float64,2}:
 44.56  27.87  37.63  17.39  50.0   …  37.63  17.39  50.0   42.96  55.74
 42.6   27.01  36.36  16.51  46.63     36.36  16.51  46.63  41.7   53.83
 39.23  19.8   31.57   4.02  39.28     31.57   4.02  39.28  36.77  49.34
 37.51   0.4   32.02  31.44  30.8      32.02  31.44  30.8   17.65  44.87
 37.18  -2.99  30.1   25.66  30.23     30.1   25.66  30.23  12.86  48.07
 36.54  -5.19  29.05  24.05  29.87  …  29.05  24.05  29.87  14.27  47.49
 35.78  -7.45  28.04  24.11  28.71     28.04  24.11  28.71  14.45  44.59

In [17]:
function charger_siting(SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices, 
        probability, P, charger_cost, opportunity_cost, alpha, beta)
    
    # Define Sets
    Γ = Array{Int}(1:length(hexagons)) # Gamma, Set with hexagons
    Σ = Array{Int}(1:length(P)) # Sigma, Set with charger types 
    Ω = Array{Int}(1:length(probability)) # Omega, Set with scenarios
    Τ = Array{Int}(1:length(hours)) # Lambda, Set with scenarios for prices

    # Initialize model    
    m = Model(CPLEX.Optimizer)
    
    # η
    @variable(m, η) 
    @variable(m, 0 <= s[k=Ω])
    β = beta # 
    α = alpha
    
    # Define variables
    @variable(m, 0 <= x_charger[i=Γ,j=Σ],Int) # number of specific charger type in each hexagon
    @variable(m, 0 <= E_supplied[i=Γ,j=Σ,k=Ω,t=Τ]) # supplied energy in each hexagon kWh
    @variable(m, 0 <= E_not_supplied[i=Γ,k=Ω])     # demand not covered kWh
    @variable(m, 0 <= cars_supplied[i=Γ,j=Σ,k=Ω],Int)
    @variable(m, 0 <= cars_not_supplied[i=Γ,j=Σ,k=Ω],Int)

    # expressions for total costs
    @expression(m, invest_cost[i=Γ,j=Σ], charger_cost[j].*x_charger[i,j].*P[j]) # first stage investment cost
    @expression(m, charging_opportunity_cost[i=Γ,k=Ω], opportunity_cost.*E_not_supplied[i,k].*probability[k])
    # @expression(m, grid_usage[i=Γ,k=Ω,t=Τ], 0.1*sum(E_supplied[i,j,k,t] for j in Σ)^(2).*probability[k])
    @expression(m, grid_usage[i=Γ,k=Ω,t=Τ], 0.1*sum(E_supplied[i,j,k,t] for j in Σ).*probability[k])
    @expression(m, charging_cost[i=Γ,j=Σ,k=Ω,t=Τ], prices[t,k].*E_supplied[i,j,k,t].*probability[k]) #minimize cost of charging
    
    # expressions for objective function
    @expression(m, total_cost, (-1)*(sum(invest_cost) + sum(charging_cost)
                                   + sum(charging_opportunity_cost) + sum(grid_usage)))
    @expression(m, cVAR, η-(1/(1-α) * dot(probability, s)))
    @objective(m, Max, (1-β)*total_cost + β*cVAR)
    
    # Constraints
    # Eq. Balance equation: Demand in each hexagon has to equal demand. Loss of load is possible
    @constraint(m, eq_balance[i=Γ,k=Ω], sum(sum(E_supplied[i,j,k,t] for j in Σ) for t in Τ) + E_not_supplied[i,k] == demand_kWh[i,k])

    # Eq. energy limit: rated_power*chargingtime must not exeed energy
    @constraint(m, eq_cap[i=Γ,j=Σ,k=Ω,t=Τ], E_supplied[i,j,k,t] <= x_charger[i,j].*P[j])

    # Eq. supplied cars and not supplied cars have to equal amount of cars in each hexagon
    @constraint(m, eq_cars_supplied[i=Γ,j=Σ,k=Ω], sum(cars_supplied[i,j,k] for j in Σ) 
                                                    + sum(cars_not_supplied[i,j,k] for j in Σ)
                                                   == demand_cars[i,k])

    # Eq. maximum one car per charging station
    @constraint(m, eq_max_cars[i=Γ,j=Σ,k=Ω], cars_supplied[i,j,k] <= x_charger[i,j])

    # Eq. 
    @constraint(m, eq_max_supply[i=Γ,j=Σ,k=Ω], sum(E_supplied[i,j,k,t] for t in Τ) <= cars_supplied[i,j,k].*car_kWh.*(1-SOC))

    # η Constraint
    @constraint(m, cVAR_constraint[k=Ω], η + (sum(charger_cost[j].*x_charger[i,j].*P[j] for j in Σ for i in Γ) 
                                           +  sum(opportunity_cost.*E_not_supplied[i,k] for i in Γ)
                                           # +  sum(0.1*sum(E_supplied[i,j,k,t] for j in Σ)^(2) for i in Γ for t in Τ)
                                           +  sum(0.1*sum(E_supplied[i,j,k,t] for j in Σ) for i in Γ for t in Τ)
                                           +  sum(prices[t,k].*E_supplied[i,j,k,t] for j in Σ for i in Γ for t in Τ))
                                           <= s[k])

    # Run optimization
    optimize!(m)

    # Evaluate resuluts
    obj = objective_value(m)
    charger_in_hex = value.(x_charger)
    res_total_cost = value.(total_cost)
    cvar = value.(cVAR)
    
    return charger_in_hex, E_not_supplied, res_total_cost, cvar
end
# x_charger, E_supplied, cars_not_supplied, cars_supplied, obj,

charger_siting (generic function with 1 method)

In [18]:
charger_in_hex, E_not_supplied, res_total_cost, cvar = charger_siting(
    SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices, 
        probability, P, charger_cost, opportunity_cost, alpha, beta)

println(charger_in_hex)
println("cvar:",cvar)
println("total_cost:",res_total_cost)

charger_amount = sum(charger_in_hex)

opportunity_cost

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value -150.216859 after 0.02 sec. (14.72 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 112032 rows and 101446 columns.
MIP Presolve modified 180 coefficients.
Reduced MIP has 14304 rows, 12960 columns, and 38256 nonzeros.
Reduced MIP has 250 binaries, 9254 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.11 sec. (109.59 ticks)
Probing time = 0.00 sec. (0.68 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 14058 rows and 12735 columns.
Reduced MIP has 246 rows, 225 columns, and 654 nonzeros.
Reduced MIP has 0 binaries, 171 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (7.45 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 246 rows, 225 columns, and 654 nonzeros.
Reduced MIP has 0 binaries, 171 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.56 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Pa

0

In [ ]:
hours_lst = [["22", "23", "24", "1", "2", "3", "4"],
            ["23", "24", "1", "2", "3", "4"],
            ["24", "1", "2", "3", "4"],
            ["1", "2", "3", "4"],
            ["2", "3", "4"],
            ["3", "4"]]
charger_hours_dict = Dict()
energy_hours_dict = Dict()
cost_hours_dict = Dict()
cvar_hours_dict = Dict()
for hours in hours_lst
    charger_in_hex, E_not_supplied, res_total_cost, cvar = charger_siting(
        SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices,
        probability, P, charger_cost, opportunity_cost, alpha, beta)
    charger_hours_dict[length(hours)] = charger_in_hex
    energy_hours_dict[length(hours)] = value.(E_not_supplied)
    cost_hours_dict[length(hours)] = res_total_cost
    cvar_hours_dict[length(hours)] = cvar
end

In [ ]:
for hours in hours_lst
    println(length(hours), " hours: ",
        "charger_in_hex = ", sum(charger_hours_dict[length(hours)]),
        " / Energy not supplied = ", sum(energy_hours_dict[length(hours)]),
        " / total cost = ", cost_hours_dict[length(hours)],
        " / Cvar = ", cvar_hours_dict[length(hours)]
    )
end

In [ ]:
for (key, value) in charger_hours_dict
    CSV.write(
        string("sensitivity/hours/charger_in_hex_",string(key),".csv"),
        DataFrame(value)
    )
end
for (key, value) in energy_hours_dict
    CSV.write(
        string("sensitivity/hours/E_not_supplied_",string(key),".csv"),
        DataFrame(value)
    )
end
CSV.write(
        string("sensitivity/hours/cost_dict.csv"),
        cost_hours_dict
    )
CSV.write(
        string("sensitivity/hours/cvar_dict.csv"),
        cvar_hours_dict
    )

In [ ]:
println(opportunity_cost)
println(hours)

In [ ]:
op_cost_lst = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
charger_op_cost_dict = Dict()
energy_op_cost_dict = Dict()
cost_op_cost_dict = Dict()
cvar_op_cost_dict = Dict()
for opportunity_cost in op_cost_lst
    charger_in_hex, E_not_supplied, res_total_cost, cvar= charger_siting(
        SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices,
        probability, P, charger_cost, opportunity_cost, alpha, beta)
    charger_op_cost_dict[opportunity_cost] = charger_in_hex
    energy_op_cost_dict[opportunity_cost] = value.(E_not_supplied)
    cost_op_cost_dict[opportunity_cost] = res_total_cost
    cvar_op_cost_dict[opportunity_cost] = cvar
end

In [ ]:
for opportunity_cost in op_cost_lst
    println(opportunity_cost, 
        " Opportunity Cost: ", "charger_in_hex = ", sum(charger_op_cost_dict[opportunity_cost]),
        " / Energy not supplied = ", sum(energy_op_cost_dict[opportunity_cost]),
        " / total cost = ", cost_op_cost_dict[opportunity_cost],
        " / Cvar = ", cvar_op_cost_dict[opportunity_cost]
    )
end

In [ ]:
for (key, value) in charger_op_cost_dict
    CSV.write(
        string("sensitivity/opportunity/charger_in_hex_",string(key),".csv"),
        DataFrame(value)
    )
end
for (key, value) in energy_op_cost_dict
    CSV.write(
        string("sensitivity/opportunity/E_not_supplied_",string(key),".csv"),
        DataFrame(value)
    )
end
CSV.write(
        string("sensitivity/opportunity/cost_dict.csv"),
        cost_op_cost_dict
    )
CSV.write(
        string("sensitivity/opportunity/cvar_dict.csv"),
        cvar_op_cost_dict
    )

In [ ]:
println(opportunity_cost)
println(hours)

In [ ]:
beta_lst = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
charger_beta_dict = Dict()
energy_beta_dict = Dict()
cost_beta_dict = Dict()
cvar_beta_dict = Dict()
for beta in beta_lst
    charger_in_hex, E_not_supplied, res_total_cost, cvar= charger_siting(
        SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices,
        probability, P, charger_cost, opportunity_cost, alpha, beta)
    charger_beta_dict[beta] = charger_in_hex
    energy_beta_dict[beta] = value.(E_not_supplied)
    cost_beta_dict[beta] = res_total_cost
    cvar_beta_dict[beta] = cvar
end

In [ ]:
for beta in beta_lst
    println(beta, 
        " beta: ", "charger_in_hex = ", sum(charger_beta_dict[beta]),
        " / Energy not supplied = ", sum(energy_beta_dict[beta]),
        " / total cost = ", cost_beta_dict[beta],
        " / Cvar = ", cvar_beta_dict[beta]
    )
end

In [ ]:
for (key, value) in charger_beta_dict
    CSV.write(
        string("sensitivity/beta/charger_in_hex_",string(key),".csv"),
        DataFrame(value)
    )
end
for (key, value) in energy_beta_dict
    CSV.write(
        string("sensitivity/beta/E_not_supplied_",string(key),".csv"),
        DataFrame(value)
    )
end
CSV.write(
        string("sensitivity/beta/cost_dict.csv"),
        cost_beta_dict
    )
CSV.write(
        string("sensitivity/beta/cvar_dict.csv"),
        cvar_beta_dict
    )

charger_cost = [600, 400, 100]# investment cost for charger 3 kW, 7 kW und 11 kW in €/kW
charger_cost_delta = 600
charger_cost_dict = Dict()
energy_dict = Dict()
for i in range(0, step=100, stop=charger_cost_delta)
    charger_in_hex, x_charger, E_not_supplied, = charger_siting(
        SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices,
        probability, P, charger_cost.+i, opportunity_cost, alpha, beta)
    charger_cost_dict[charger_cost.+i] = charger_in_hex
    energy_dict[charger_cost.+i] = value.(E_not_supplied)
end

for key in keys(charger_cost_dict)
    println(key, " Charger Cost: ", "charger_in_hex = ", sum(charger_cost_dict[key]),
        " / Energy not supplied = ", sum(energy_dict[key]))
end